In [1]:

train_msg = open("../../MCMD/java/sort_random_train80_valid10_test10/train.msg.txt", encoding='UTF-8').read().split("\n")
len(train_msg)

In [7]:
valid_msg = open("../../MCMD/java/sort_random_train80_valid10_test10/valid.msg.txt", encoding='UTF-8').read().split("\n")
len(valid_msg)

45001

In [9]:
test_msg = open("../../MCMD/java/sort_random_train80_valid10_test10/test.msg.txt", encoding='UTF-8').read().split("\n")
len(test_msg)

45001

## Preprocess script

In [41]:
import re
def process_diff(diff_text):
    # Replace <nl> with \n
    diff_text = diff_text.replace('<nl>', '\n')
    diff_text = re.sub(r'(?<=\S)\s*([^\w\s])\s*(?=\S)', r'\1', diff_text)
    diff_text = re.sub(r'\n\s+', r'\n', diff_text)
    return diff_text

def process_msg(msg_text):
    msg_text = re.sub(r'(?<=\S)\s*([^\w\s])\s*(?=\S)', r'\1', msg_text)
    return msg_text

In [58]:
import json
def extract_diffs(indices, diff_file_path):
    diffs = []
    with open(diff_file_path, 'r', encoding='UTF-8') as f:
        for i, line in enumerate(f):
            if i in indices:
                diffs.append(process_diff(line))
    return diffs


selected_messages = []
selected_languages = []
selected_diffs = []
train_msg_path = ["../../MCMD/java/sort_random_train80_valid10_test10/train.msg.txt", "../../MCMD/cpp/sort_random_train80_valid10_test10/train.msg.txt", "../../MCMD/csharp/sort_random_train80_valid10_test10/train.msg.txt", "../../MCMD/python/sort_random_train80_valid10_test10/train.msg.txt", "../../MCMD/javascript/sort_random_train80_valid10_test10/train.msg.txt"]
train_diff_path = ["../../MCMD/java/sort_random_train80_valid10_test10/train.diff.txt", "../../MCMD/cpp/sort_random_train80_valid10_test10/train.diff.txt", "../../MCMD/csharp/sort_random_train80_valid10_test10/train.diff.txt", "../../MCMD/python/sort_random_train80_valid10_test10/train.diff.txt", "../../MCMD/javascript/sort_random_train80_valid10_test10/train.diff.txt"]

for msg_path, diff_path in zip(train_msg_path, train_diff_path):
    selected_indices = []
    with open(msg_path, 'r', encoding='UTF-8') as f:
        count = 0
        for i, line in enumerate(f):
            if count>= 1000:
                break
            # line = process_msg(line)
            if len(line.split()) >= 5:
                selected_indices.append(i)
                selected_messages.append(line.strip())
                selected_languages.append(msg_path.split('/')[3])
                count += 1
            
        selected_diffs.extend(extract_diffs(selected_indices, diff_path))

# Create a list of dictionaries containing selected messages and diffs
data = []
for msg, diff, language in zip(selected_messages, selected_diffs, selected_languages):
    item = {
        'msg': msg,
        'diff': diff,
        'language': language
    }
    data.append(item)

# Write the data to a JSON file
with open('data/selected_data.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

# # Write selected messages to a new file
# with open('dataset/selected_messages.txt', 'w', encoding='UTF-8') as f:
#     for msg in selected_messages:
#         f.write(msg + '\n')  # Add a newline after each message
# 
# # Write selected diffs to a new file
# with open('dataset/selected_diffs.txt', 'w', encoding='UTF-8') as f:
#     for diff in selected_diffs:
#         f.write(diff)  # Add a newline after each diff